In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPool2D, TimeDistributed, BatchNormalization, SimpleRNN, Reshape, LSTM, Permute
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

## Useful functions

In [64]:
def save_model(model, filename):
    model_json = model.to_json()
    with open(filename + ".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(filename + ".h5")
    print("Saved model to disk")

## Load data

In [65]:
DATA_FOLDER = "./data/"

In [66]:
df_training = pd.read_parquet(DATA_FOLDER + "training.parquet")

In [67]:
df_public_test = pd.read_parquet(DATA_FOLDER + "public_test.parquet")

In [68]:
df_private_test = pd.read_parquet(DATA_FOLDER + "private_test.parquet")

## Process data

In [69]:
x_train = df_training.as_matrix()[:,1:]

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [70]:
x_train = df_training.as_matrix()[:,1:]
y_train = df_training.as_matrix()[:,0]
x_test = df_public_test.as_matrix()[:,1:]
y_test = df_public_test.as_matrix()[:,0]

E:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
E:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.


In [71]:
x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)

In [72]:
x_train = x_train.astype("float32")/255.
x_test = x_test.astype("float32")/255.

In [73]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Model

In [113]:
model = Sequential()

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu',
                 input_shape = (48, 48, 1)))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPool2D(strides=(2,2)))
model.add(Dropout(0.25))

model.add(Permute((3, 2, 1)))
model.add(Reshape((64, 81)))
model.add(LSTM(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Reshape((128, 2)))
model.add(Flatten())
model.add(Dense(7, activation='softmax'))

In [114]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_157 (Conv2D)          (None, 46, 46, 64)        640       
_________________________________________________________________
batch_normalization_155 (Bat (None, 46, 46, 64)        256       
_________________________________________________________________
conv2d_158 (Conv2D)          (None, 44, 44, 64)        36928     
_________________________________________________________________
batch_normalization_156 (Bat (None, 44, 44, 64)        256       
_________________________________________________________________
max_pooling2d_77 (MaxPooling (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_118 (Dropout)        (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_159 (Conv2D)          (None, 20, 20, 64)        36928     
__________

In [115]:
datagen = ImageDataGenerator(zoom_range = 0.1,
                            height_shift_range = 0.1,
                            width_shift_range = 0.1,
                            rotation_range = 10)

In [116]:
model.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-4), metrics=["accuracy"])

In [117]:
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [118]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=500,
                           epochs=20, #Increase this when not on Kaggle kernel
                           validation_data=(x_test[:400,:], y_test[:400,:]), #For speed
                           callbacks=[annealer])

Epoch 1/20
500/500 [==============================] - 742s 1s/step - loss: 1.8241 - acc: 0.2542 - val_loss: 1.6952 - val_acc: 0.3375
Epoch 2/20
500/500 [==============================] - 735s 1s/step - loss: 1.7196 - acc: 0.3028 - val_loss: 1.7646 - val_acc: 0.3275
Epoch 3/20
500/500 [==============================] - 746s 1s/step - loss: 1.6701 - acc: 0.3412 - val_loss: 1.6557 - val_acc: 0.3475
Epoch 4/20
500/500 [==============================] - 716s 1s/step - loss: 1.6128 - acc: 0.3639 - val_loss: 1.5729 - val_acc: 0.4000
Epoch 5/20
500/500 [==============================] - 724s 1s/step - loss: 1.5681 - acc: 0.3878 - val_loss: 1.4605 - val_acc: 0.4425
Epoch 6/20
500/500 [==============================] - 759s 2s/step - loss: 1.5427 - acc: 0.4005 - val_loss: 1.3864 - val_acc: 0.4525
Epoch 7/20
500/500 [==============================] - 783s 2s/step - loss: 1.5087 - acc: 0.4108 - val_loss: 1.3632 - val_acc: 0.4900
Epoch 8/20
500/500 [==============================] - 814s 2s/step - 

In [120]:
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                           steps_per_epoch=500,
                           initial_epoch=20,
                           epochs=40, #Increase this when not on Kaggle kernel
                           validation_data=(x_test[:400,:], y_test[:400,:]), #For speed
                           callbacks=[annealer])

Epoch 21/40
500/500 [==============================] - 731s 1s/step - loss: 1.2881 - acc: 0.5018 - val_loss: 1.1696 - val_acc: 0.5975
Epoch 22/40
500/500 [==============================] - 736s 1s/step - loss: 1.2911 - acc: 0.5024 - val_loss: 1.1683 - val_acc: 0.5850
Epoch 23/40
500/500 [==============================] - 748s 1s/step - loss: 1.2942 - acc: 0.5030 - val_loss: 1.1738 - val_acc: 0.5775
Epoch 24/40
500/500 [==============================] - 725s 1s/step - loss: 1.2839 - acc: 0.5088 - val_loss: 1.1568 - val_acc: 0.5950
Epoch 25/40
500/500 [==============================] - 705s 1s/step - loss: 1.2815 - acc: 0.5088 - val_loss: 1.1614 - val_acc: 0.5900
Epoch 26/40
500/500 [==============================] - 711s 1s/step - loss: 1.2871 - acc: 0.5054 - val_loss: 1.1535 - val_acc: 0.5975
Epoch 27/40
500/500 [==============================] - 707s 1s/step - loss: 1.2705 - acc: 0.5098 - val_loss: 1.1504 - val_acc: 0.6000
Epoch 28/40
500/500 [==============================] - 710s 1s

In [ ]:
final_loss, final_acc = model.evaluate(x_test, y_test, verbose=0)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [121]:
save_model(model, "./trained_models/elena_model1.1")

Saved model to disk


In [ ]:
def plot_history(history):
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

In [ ]:
plot_history(hist)